In [2]:
# 查看数据文件目录  list datalab files
!ls

datalab  myspace  work


In [3]:
# 查看个人永久空间文件  list files in your permanent storage
!ls /home/tianchi/myspace/


1  result.csv


In [4]:
import json
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV  # Perforing grid search


/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [5]:
df_train = pd.read_csv('datalab/37693/first_round_training_data.csv')
df_test = pd.read_csv('datalab/37693/first_round_testing_data.csv')

In [6]:
dit = {'Excellent':0,'Good':1,'Pass':2,'Fail':3}
df_train['label'] = df_train['Quality_label'].map(dit)
# 将特征转换成数字

In [7]:
df_train=df_train.drop(labels=['Attribute1','Attribute2','Attribute3','Attribute4','Attribute5','Attribute6','Attribute7',
                       'Attribute8','Attribute9','Attribute10','Quality_label', ],axis=1)

In [8]:
data=df_train.drop(labels='label',axis=1)
target = df_train['label']
X_train,X_test,y_train,y_test =train_test_split(data,target,test_size=0.2)

In [20]:

# 创建成lgb特征的数据集格式
lgb_train = lgb.Dataset(X_train, y_train) # 将数据保存到LightGBM二进制文件将使加载更快
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)  # 创建验证数据
 
# 将参数写成字典下形式
params = {  
    'boosting_type': 'gbdt',  
    'objective': 'multiclass',  
    'num_class': 4,  
    'metric': 'multi_error',  
    'num_leaves': 120,  
    'min_data_in_leaf': 100,  
    'learning_rate': 0.00007,  
    'feature_fraction': 0.8,  
    'bagging_fraction': 0.8,  
    'bagging_freq': 5,  
    'lambda_l1': 0.4,  
    'lambda_l2': 0.5,  
    'min_gain_to_split': 0.2,  
    'verbose': -1, 
}
gbm = lgb.train(params,lgb_train,num_boost_round=30000,valid_sets=lgb_eval,early_stopping_rounds=5000,verbose_eval = 1000) # 训练数据需要参数列表和数据集
#  改的学习率和num_leave最好都是0.4775


Training until validation scores don't improve for 5000 rounds.
[1000]	valid_0's multi_error: 0.486667
[2000]	valid_0's multi_error: 0.486667
[3000]	valid_0's multi_error: 0.49
[4000]	valid_0's multi_error: 0.490833
[5000]	valid_0's multi_error: 0.4925
Early stopping, best iteration is:
[111]	valid_0's multi_error: 0.4775


In [10]:
example = pd.read_csv('datalab/37693/submit_example.csv')

In [11]:
prediction=gbm.predict(df_test.drop(labels='Group',axis=1))
# prediction

In [12]:
prediction.shape

(6000, 4)

In [13]:
df_test['Excellent ratio'] = prediction[:,0]
df_test['Good ratio'] = prediction[:,1]
df_test['Pass ratio'] = prediction[:,2]
df_test['Fail ratio'] = prediction[:,3]
result = pd.DataFrame({'Group':example.Group})

In [14]:
for group_num in range(120):
#     print(group_num)
#     all_counts = df_test.loc[df_test['Group'] ==group_num]['Excellent ratio'].mean()
#     print(all_counts)
    try:
        result.loc[result['Group'] ==group_num,'Excellent ratio'] = df_test.loc[df_test['Group'] ==group_num]['Excellent ratio'].mean()
    except Exception as e:
        result.loc[result['Group'] ==group_num,'Excellent ratio'] = 0
    try:
        result.loc[result['Group'] ==group_num,'Good ratio'] = df_test.loc[df_test['Group'] ==group_num]['Good ratio'].mean()
    except Exception as e:
        result.loc[result['Group'] ==group_num,'Good ratio'] = 0
    try:
        result.loc[result['Group'] ==group_num,'Pass ratio'] = df_test.loc[df_test['Group'] ==group_num]['Pass ratio'].mean()
    except Exception as e:
        result.loc[result['Group'] ==group_num,'Pass ratio'] = 0
    try:
        result.loc[result['Group'] ==group_num,'Fail ratio'] = df_test.loc[df_test['Group'] ==group_num]['Fail ratio'].mean()
    except Exception as e:
        result.loc[result['Group'] ==group_num,'Fail ratio'] = 0

In [15]:
result.to_csv(r'/home/tianchi/myspace/result.csv',index=False)